In [4]:
import pandas as pd

df = pd.read_csv(r'd:\naomy\LIA-FastAPI-MySQL\data\data.csv')
df.head()

,id,title,author,description,genre,classification,pages,owner_id
0,1,Matadouro cinco,Kurt Vonnegut,Um livro que zomba da estupidez humana com um ...,Science Fiction,Excellent,288,1
1,2,Cama de gato,Kurt Vonnegut,Mais um livro incrível que zomba da condição h...,Science Fiction,Excellent,280,1
2,3,Cem anos de solidão,Gabriel Garcia Marquez,Um realismo fantástico lindo e cativante,Fiction Novel,Excellent,448,1
3,4,Sobre os ossos dos mortos,Olga Tockarzuck,Uma história sobre nossa relação com a naturez...,Fiction Novel,Excellent,256,1
4,5,Ensaio sobre a cegueira,José Saramago,É preciso que a humanidade deixe de ser cega,Fiction Novel,Excellent,312,1
